In [404]:
with open('day_05.txt') as f:
    data = [int(token) for token in f.readlines()[0].strip().split(',')]

# Setup
For this problem I am trying an OOP approach, where each operation is a service object that inherits from a parent `Instruction` class. 

Come along with me now.

First we define some constants to make things more human readable later. Here are the IDs corresponding to each opcode, the lengths of each instruction type, and indicators for mode types.

In [405]:
# Opcode IDs
ADD = 1
MULTIPLY = 2
INPUT = 3
OUTPUT = 4
JUMP_IF_TRUE = 5
JUMP_IF_FALSE = 6
LESS_THAN = 7
EQUALS = 8
HALT = 99

In [406]:
# Opcode param lengths
NUM_PARAMS = {
    ADD: 3,
    MULTIPLY: 3,
    INPUT: 1,
    OUTPUT: 1,
    JUMP_IF_TRUE: 2,
    JUMP_IF_FALSE: 2,
    LESS_THAN: 3,
    EQUALS: 3,
    HALT: 0,
}

In [407]:
# Opcode param modes
POSITION_MODE = 0
IMMEDIATE_MODE = 1

# Classes
It's class time. 

Here we define the `Instruction` class, which contains all the info for a single instruction:
* the opcode and its length
* the list of params for the opcode
* the list of modes for each param
* the entire state of current memory

We also define a method for fetching a value from memory (including mode stuff), and writing a value to memory.

In [408]:
class Instruction:
    def __init__(self, opcode, length, params, modes, memory):
        self.opcode = opcode
        self.length = length
        self.params = params
        self.modes = modes
        self.memory = memory
        
    def get_val(self, param_index):
        if self.modes[param_index] == IMMEDIATE_MODE:
            return self.params[param_index]
        elif self.modes[param_index] == POSITION_MODE: 
            return self.memory[self.params[param_index]]
    
    def put_val(self, param_address_index, value):
        self.memory[self.params[param_address_index]] = value        

The one thing the `Instruction` class doesn't do is to execute the opcodes' operations. That's where the inherited service object classes come in. 

Here we define a subclass of `Instruction` for each operation, each with a single `call()` method. This method uses the parent methods for getting values from memory and setting the result.

In [409]:
class Add(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        self.put_val(2, val0 + val1)

In [410]:
class Multiply(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        self.put_val(2, val0 * val1)   

The `Input` class needs an input value. We do this just by defining a global constant called `INPUT_VALUE`. Why not!

In [411]:
INPUT_VALUE = 1 #default
class Input(Instruction):
    def call(self):
        self.put_val(0, INPUT_VALUE)

In [412]:
class Output(Instruction):
    def call(self):
        val = self.get_val(0)
        print("Output:", val)

The two Jump classes, `JumpIfTrue` and `JumpIfFalse`, are responsible for setting the memory pointer directly, rather than simply proceeding to the next sequential `Instruction`. We accomplish this by returning the value the pointer should be set to, so the caller can reset that pointer accordingly.

In [413]:
class JumpIfTrue(Instruction):
    def call(self):
        in_val = self.get_val(0)
        out_val = self.get_val(1)
        if in_val:
            return out_val

In [414]:
class JumpIfFalse(Instruction):
    def call(self):
        in_val = self.get_val(0)
        out_val = self.get_val(1)
        if not in_val:
            return out_val        

In [415]:
class LessThan(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        out_val = 1 if val0 < val1 else 0
        self.put_val(2, out_val)           

In [416]:
class Equals(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        out_val = 1 if val0 == val1 else 0
        self.put_val(2, out_val)

The `Halt` instruction doesn't do anything, but we let it have its own class anyway.

In [417]:
class Halt(Instruction):
    def call(self):
        pass

# Instruction factory
Here we define a dictionary that maps opcodes to operation classes, and a method `create_instruction` that parses the instruction encoded at a given memory location and creates the corresponding `Instruction` subclass.

I think this is maybe some kind of factory pattern?

In [418]:
OPS = {
    ADD: Add,
    MULTIPLY: Multiply,
    INPUT: Input,
    OUTPUT: Output,
    JUMP_IF_TRUE: JumpIfTrue,
    JUMP_IF_FALSE: JumpIfFalse,
    LESS_THAN: LessThan,
    EQUALS: Equals,
    HALT: Halt,
}

In [419]:
def create_instruction(memory, memory_pointer):
    encoded_opcode = memory[memory_pointer]
    opcode = int(str(encoded_opcode)[-2:])
    
    num_params = NUM_PARAMS[opcode]
    
    reverse_modes = str(encoded_opcode)[:-2][::-1]
    modes = [int(char) for char in reverse_modes] + [0] * (num_params - len(reverse_modes))
    
    params = memory[memory_pointer + 1 : memory_pointer + num_params + 1]
    
    return OPS[opcode](opcode, num_params + 1, params, modes, memory)

# Running the program
Finally, we can wrap all this stuff up into a `Program` class that executes the whole Intcode sequence. This class just has a `run` method that creates each instruction, calls it, and repeats until we reach the `HALT` instruction.

Enjoy!

In [420]:
class Program:
    def __init__(self, initial_memory):
        self.memory = initial_memory[:]
        self.memory_pointer = 0
        
    def run(self):
        while(1):
            instruction = create_instruction(self.memory, self.memory_pointer)
            if instruction.opcode == HALT:
                break
                
            jump_pointer = instruction.call()
            self.memory_pointer = jump_pointer or (self.memory_pointer + instruction.length)

## Part 1
Here we set the `INPUT_VALUE` to 1, and the Intcode program does some diagnostics that only uses `Add`, `Multiply`, `Input`, `Output`, and `Halt` instructions.

In [423]:
INPUT_VALUE = 1
Program(data).run()

Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 5074395


## Part 2
For Part 2, we set the `INPUT_VALUE` to 5, and that executes a different program that includes all the opcode instructions. Fun!

In [424]:
INPUT_VALUE = 5
Program(data).run()

Output: 8346937
